In [1]:
"""
Read VIIRS active fire data
"""

# Import packages
import os, time, glob
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rioxarray as rxr
import h5py
import pyproj
import xarray as xr
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from affine import Affine
from osgeo import gdal, gdal_array, gdalconst, osr

# Projection information
geog = 'EPSG:4326'  # Geographic projection
prj = 'EPSG:5070'  # Projected coordinate system- WGS 84 NAD83 UTM Zone 13N

# File path information
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/'

# File path information
print("Success")

Success


In [4]:
# Function to list files in a directory
def list_files(path, ext, recursive):
    """
    List files of a specific type in a directory or subdirectories
    """
    if recursive is True:
        return glob.glob(os.path.join(path, '**', '*{}'.format(ext)), recursive=True)
    else:
        return glob.glob(os.path.join(path, '*{}'.format(ext)), recursive=False)

# Get a list of .nc files
nc_files = list_files(os.path.join(maindir, 'data/spatial/raw/VIIRS'), "*.nc", recursive=True)
h5_files = list_files(os.path.join(maindir, 'data/spatial/raw/VIIRS'), "*.h5", recursive=True)

# Seperate into VNP14 and VJ114, VNP03/VJ103

vnp = [f for f in nc_files if 'VNP14' in os.path.basename(f)] # VNP14IMG
vji = [f for f in nc_files if 'VJ114' in os.path.basename(f)] # VJI14IMG

print(f'VNP14IMG: {len(vnp)} granules')
print(f'VJI14IMG: {len(vji)} granules')


VNP14IMG: 158 granules
VJI14IMG: 155 granules


In [5]:
# Split the path
parts = vnp[0].split("/")
for part in parts:
    if part.startswith("FIRED_"):
        fire_id = part
        break
print(fire_id)

FIRED_4014


In [6]:
vnp[0]

'/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/VIIRS/FIRED_4014/VNP14IMG/VNP14IMG.A2019225.1936.002.2024095014143.nc'

In [7]:
# Library to work with netCDF files
from netCDF4 import Dataset

# Open a .nc file ("file_name")
ds = Dataset(vnp[0], 'r', format = 'NETCDF4')

print(ds)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    VNP02IMG: VNP02CCIMG.A2019225.1936.002.2022333062425.nc
    VNP03IMG: VNP03IMG.A2019225.1936.002.2021110215728.nc
    VNP02MOD: VNP02CCMOD.A2019225.1936.002.2022333062425.nc
    VNP02GDC: VNP02GDC.A2019225.1936.002.2021111141115.nc
    ProcessVersionNumber: 3.1.9
    ExecutableCreationDate: Oct 31 2023
    ExecutableCreationTime: 17:46:00
    SystemID: Linux minion20367 5.4.0-1093-fips #103-Ubuntu SMP Thu Feb 8 14:02:37 UTC 2024 x86_64
    Unagg_GRingLatitude: 51.740856,59.026741,38.287682,33.210144
    Unagg_GRingLongitude: -129.983383,-81.693024,-81.958115,-115.527855
    NorthBoundingCoordinate: 59.20688
    SouthBoundingCoordinate: 33.210144
    EastBoundingCoordinate: -81.69302
    WestBoundingCoordinate: -129.98338
    Day/Night/Both: Day
    FirePix: 63
    DayPix: 41369600
    LandPix: 39874606
    Satellite: NPP
    NightPix: 0
    DayNightFlag: Day
    WaterPix: 1494994
    MissingPix: 9

In [8]:
day_night_flag = ds.getncattr('DayNightFlag')
print(day_night_flag)

Day


In [9]:
# Print a list of variables in file
list(ds.variables)

['FP_AdjCloud',
 'FP_AdjWater',
 'FP_MAD_DT',
 'FP_MAD_T4',
 'FP_MAD_T5',
 'FP_MeanDT',
 'FP_MeanRad13',
 'FP_MeanT4',
 'FP_MeanT5',
 'FP_Rad13',
 'FP_SolAzAng',
 'FP_SolZenAng',
 'FP_T4',
 'FP_T5',
 'FP_ViewAzAng',
 'FP_ViewZenAng',
 'FP_WinSize',
 'FP_confidence',
 'FP_day',
 'FP_latitude',
 'FP_line',
 'FP_longitude',
 'FP_power',
 'FP_sample',
 'algorithm QA',
 'fire mask']

In [13]:
ds.getncattr('PlatformShortName')

'SUOMI-NPP'

In [ ]:
print(ds.variables[''])

In [ ]:
print(ds.variables['FP_confidence'][:])

In [ ]:
# Extract the acquisition date and time from the NetCDF file
start_time_str = ds.getncattr('PGE_StartTime')
acq_datetime = datetime.strptime(start_time_str, '%Y-%m-%d %H:%M:%S.%f')
# Calculate Julian Day
julian_day = acq_datetime.timetuple().tm_yday

print(f'Acquisition date/time: {acq_datetime}')
print(f'Acquisition Julian Day: {julian_day}')

In [ ]:
# Extract fire pixel information
latitudes = ds.variables['FP_latitude'][:]
longitudes = ds.variables['FP_longitude'][:]
fire_radiative_power = ds.variables['FP_power'][:]
fire_confidence = ds.variables['FP_confidence'][:]
day_night = ds.variables['FP_day'][:]

# Create a DataFrame with the fire pixel data
fire_data = pd.DataFrame({
    'latitude': latitudes,
    'longitude': longitudes,
    'fire_radiative_power': fire_radiative_power,
    'fire_confidence': fire_confidence,
    'day_night': day_night,
    'acq_datetime': acq_datetime,
    'acq_julian_day': julian_day
})

fire_data.head()

In [ ]:
print(len(fire_data))

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a Folium map centered around the mean latitude and longitude
m = folium.Map(location=[fire_data['latitude'].mean(), fire_data['longitude'].mean()], zoom_start=6)

# Add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(m)

# Add markers to the map
for idx, row in fire_data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f'FRP: {row["fire_radiative_power"]}',
        icon=folium.Icon(color='red')
    ).add_to(marker_cluster)

# Display the map
m.save('fire_radiative_power_map.html')
m